# Libraries import
Importing the necessary libraries for work.

`yfinance`: Allows you to download historical stock market data and financials from Yahoo Finance.

`pandas`: Provides powerful data structures like DataFrames for data analysis and manipulation.

`matplotlib.pyplot`: A plotting library used to create basic visualizations like line and bar charts.

`seaborn`: Builds on Matplotlib to create more attractive and informative statistical plots.

`matplotlib.gridspec`: Enables advanced control over subplot layouts within a figure.

`sqlite3`: Lets you create and interact with local SQLite databases using SQL queries in Python

In [ ]:

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import sqlite3

# Downloading data

Downloading the data for the cryptocurrencies from yfinance for period from 2022 to 2025

In [ ]:
BTC = yf.download("BTC-USD", start="2022-01-01", end="2025-01-01")["Close"]
ETH = yf.download("ETH-USD", start="2022-01-01", end="2025-01-01")["Close"]
SOL = yf.download("SOL-USD", start="2022-01-01", end="2025-01-01")["Close"]


/tmp/ipython-input-3295612328.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  BTC = yf.download("BTC-USD", start="2022-01-01", end="2025-01-01")["Close"]
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3295612328.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ETH = yf.download("ETH-USD", start="2022-01-01", end="2025-01-01")["Close"]
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3295612328.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  SOL = yf.download("SOL-USD", start="2022-01-01", end="2025-01-01")["Close"]
[*********************100%***********************]  1 of 1 completed


# Combining evrything

Combining every cryptocurrency into one DataFrame, so then i can work with it in SQL

In [ ]:
df = pd.concat([BTC, ETH, SOL], axis=1)
df.columns = ["BTC", "ETH", "SOL"]
print(df)

                     BTC          ETH         SOL
Date                                             
2022-01-01  47686.812500  3769.697021  178.517944
2022-01-02  47345.218750  3829.564941  176.382843
2022-01-03  46458.117188  3761.380371  170.297745
2022-01-04  45897.574219  3794.056641  167.938904
2022-01-05  43569.003906  3550.386963  155.099731
...                  ...          ...         ...
2024-12-27  94164.859375  3328.916992  183.828476
2024-12-28  95163.929688  3397.902344  195.013535
2024-12-29  93530.226562  3349.513428  189.744263
2024-12-30  92643.210938  3356.392578  191.028046
2024-12-31  93429.203125  3332.531738  189.264481

[1096 rows x 3 columns]


# Connecting to SQL

Connecting to temporary sql server using sqlite3,connect, resetting the index so the `Date` column is not dissapearing

In [ ]:
conn=sqlite3.connect(":memory:")
df_reset = df.reset_index()
df_reset.to_sql("Crypto", conn, index=False, if_exists="replace")


1096

# SQL part

Now, calculating the daily return with `LAG` Function

Example:
```python
(BTC - LAG(BTC) OVER (ORDER BY Date)) / LAG(BTC) OVER (ORDER BY Date) AS BTC_return
```
can also be read as
```python
(Today`s Price-Yesterday`s Price)/Yesterday`s Price
```

Doing it for every cryptocurrency.

In [ ]:
query="""
WITH daily_returns AS(
  SELECT
     Date,
     BTC,
     ETH,
     SOL,
     (BTC - LAG(BTC) OVER (ORDER BY Date)) / LAG(BTC) OVER (ORDER BY Date) AS BTC_return,
     (ETH - LAG(ETH) OVER (ORDER BY Date)) / LAG(ETH) OVER (ORDER BY Date) AS ETH_return,
     (SOL - LAG(SOL) OVER (ORDER BY Date)) / LAG(SOL) OVER (ORDER BY Date) AS SOL_return
     FROM Crypto
)
SELECT *
FROM daily_returns
WHERE BTC_return IS NOT NULL;
"""
data = pd.read_sql_query(query, conn)
print(data)

                     Date           BTC          ETH         SOL  BTC_return  \
0     2022-01-02 00:00:00  47345.218750  3829.564941  176.382843   -0.007163   
1     2022-01-03 00:00:00  46458.117188  3761.380371  170.297745   -0.018737   
2     2022-01-04 00:00:00  45897.574219  3794.056641  167.938904   -0.012066   
3     2022-01-05 00:00:00  43569.003906  3550.386963  155.099731   -0.050734   
4     2022-01-06 00:00:00  43160.929688  3418.408203  150.431351   -0.009366   
...                   ...           ...          ...         ...         ...   
1090  2024-12-27 00:00:00  94164.859375  3328.916992  183.828476   -0.017022   
1091  2024-12-28 00:00:00  95163.929688  3397.902344  195.013535    0.010610   
1092  2024-12-29 00:00:00  93530.226562  3349.513428  189.744263   -0.017167   
1093  2024-12-30 00:00:00  92643.210938  3356.392578  191.028046   -0.009484   
1094  2024-12-31 00:00:00  93429.203125  3332.531738  189.264481    0.008484   

      ETH_return  SOL_return  
0       

Then, moving to excel where i will calculate the metrics